In [1]:
pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16  # You can use a pre-trained model for feature extraction


In [3]:
image_dir = r"C:\Users\Kanishk Goyal\OneDrive - IIT Kanpur\Desktop\image_test_data"
csv_file = r"C:\Users\Kanishk Goyal\OneDrive - IIT Kanpur\Desktop\deep features_100_images (1).csv"

In [4]:
df = pd.read_csv(csv_file)
image_paths = [os.path.join(image_dir, img_name) for img_name in df['Image Name']]
income_values = df['Income'].values

In [5]:
# Define a CNN model for regression
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1))  # Output layer for regression

In [6]:
model.compile(optimizer='adam', loss='mean_squared_error')


In [7]:
X = []
for img_path in image_paths:
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.applications.vgg16.preprocess_input(img)  # Preprocess according to the chosen model (VGG16, in this case)
    X.append(img)

In [8]:
X = np.array(X)
y = income_values

In [9]:
model.fit(X, y, epochs=10, batch_size=32)


Epoch 1/10
4/4 [==============================] - 4s 587ms/step - loss: 98241183553159168.0000
Epoch 2/10
4/4 [==============================] - 2s 512ms/step - loss: 98238168486117376.0000
Epoch 3/10
4/4 [==============================] - 2s 517ms/step - loss: 98229638681067520.0000
Epoch 4/10
4/4 [==============================] - 2s 510ms/step - loss: 98219777436155904.0000
Epoch 5/10
4/4 [==============================] - 2s 513ms/step - loss: 98174998107127808.0000
Epoch 6/10
4/4 [==============================] - 2s 496ms/step - loss: 98092964231774208.0000
Epoch 7/10
4/4 [==============================] - 2s 492ms/step - loss: 98039929975603200.0000
Epoch 8/10
4/4 [==============================] - 2s 488ms/step - loss: 97796130452013056.0000
Epoch 9/10
4/4 [==============================] - 2s 499ms/step - loss: 97559993150078976.0000
Epoch 10/10
4/4 [==============================] - 2s 498ms/step - loss: 97336861009117184.0000


In [10]:
model.save('income_cnn_model.h5')

c:\Users\Kanishk Goyal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
feature_extractor_model = Model(inputs=model.input, outputs=model.layers[-2].output)


In [12]:
deep_features = feature_extractor_model.predict(X)

4/4 [==============================] - 1s 122ms/step


In [13]:
deep_features_df = pd.DataFrame(deep_features)

In [14]:
deep_features_df

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.0,0.0,2683132.500,0.0,0.0,0.0,2680161.250,2685690.000,2685542.50,2652478.50,...,2688242.00,2680969.50,2681428.00,0.0,2693779.500,0.0,0.0,2685898.250,0.0,0.0
1,0.0,0.0,3867436.750,0.0,0.0,0.0,3863443.500,3871173.500,3870173.00,3823834.50,...,3874145.50,3864327.00,3864776.25,0.0,3882984.250,0.0,0.0,3871694.750,0.0,0.0
2,0.0,0.0,1721867.125,0.0,0.0,0.0,1720748.875,1723720.875,1722524.75,1702582.00,...,1724996.50,1720874.25,1720350.25,0.0,1728915.875,0.0,0.0,1723598.125,0.0,0.0
3,0.0,0.0,3399957.500,0.0,0.0,0.0,3395827.250,3403438.500,3403186.00,3361515.50,...,3406577.75,3397484.00,3397994.75,0.0,3412961.500,0.0,0.0,3403860.000,0.0,0.0
4,0.0,0.0,3456809.000,0.0,0.0,0.0,3452727.250,3460376.000,3459676.25,3417186.00,...,3463711.75,3453891.50,3454595.00,0.0,3470374.250,0.0,0.0,3460503.500,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,0.0,0.0,3317371.000,0.0,0.0,0.0,3313870.500,3321056.500,3320109.75,3279854.50,...,3324260.50,3315185.75,3314960.00,0.0,3329878.000,0.0,0.0,3320419.500,0.0,0.0
94,0.0,0.0,3443044.250,0.0,0.0,0.0,3439748.500,3445917.000,3445793.00,3404056.00,...,3448951.50,3440147.50,3439992.50,0.0,3456415.500,0.0,0.0,3446662.750,0.0,0.0
95,0.0,0.0,1471565.750,0.0,0.0,0.0,1470338.375,1473114.875,1472806.50,1454959.75,...,1474603.50,1470775.75,1470791.00,0.0,1477406.625,0.0,0.0,1473146.500,0.0,0.0
96,0.0,0.0,3724212.000,0.0,0.0,0.0,3720188.000,3728079.250,3727452.00,3682078.25,...,3731828.00,3721638.25,3721966.00,0.0,3738876.250,0.0,0.0,3728580.250,0.0,0.0
